<a href="https://colab.research.google.com/github/sgundala/RAG-from-Scratch/blob/main/RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if "COLAB_GPU" in os.environ:
  print("running on Colab GPU")
  !pip install PyMuPDF
  !pip install torch
  !pip install tqdm
  !pip install sentence-transformers
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation

running on Colab GPU
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 141.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
!nvidia-smi


Wed Nov 12 15:05:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvidia-smi || echo "No GPU found"

Wed Nov 12 15:05:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import fitz

In [ ]:
from tqdm.auto import tqdm

In [ ]:
PDF_PATH = "/content/Human-Nutrition.pdf"  # update if your filename differs


In [ ]:
import fitz  # PyMuPDF
from tqdm.auto import tqdm
import re

def text_formatter(text: str) -> str:
    if not text:
        return ""
    # keep it simple: flatten whitespace
    text = text.replace("\r", "\n").replace("\t", " ")
    text = " ".join(text.split()).strip()
    return text

def open_and_read_pdf(pdf_path: str, skip_first_pages: int = 41):
    """
    Returns a list of dicts, one per kept page:
      {
        "Page_number": <int>,                 # starts at 1 after skipping
        "Page_char_count": <int>,
        "page_word_count": <int>,
        "page_sentence_count_raw": <int>,     # split on . ! ?
        "page_token_count": <int>,            # ~heuristic: 1 token ≈ 4 chars
        "text": <str>
      }
    """
    doc = fitz.open(pdf_path)
    pages_and_texts = []

    # enumerate from 1 to match human page numbers
    for i, page in tqdm(enumerate(doc, start=1), total=len(doc), desc="Reading pages"):
        # skip front matter
        if i <= skip_first_pages:
            continue

        text = page.get_text("text")
        text = text_formatter(text)

        # simple counts (fast, no extra deps)
        char_count = len(text)
        word_count = len(text.split())
        sent_count = len([s for s in re.split(r"[.!?]+", text) if s.strip()])
        token_count = char_count // 4  # heuristic; swap for tiktoken if you like

        pages_and_texts.append({
            "Page_number": i - skip_first_pages,     # adjusted: first kept page -> 1
            "Page_char_count": char_count,
            "page_word_count": word_count,
            "page_sentence_count_raw": sent_count,
            "page_token_count": token_count,
            "text": text
        })

    doc.close()
    return pages_and_texts

# ---- run it ----
pdf_path = "/content/Human-Nutrition.pdf"
pages_and_texts = open_and_read_pdf(pdf_path, skip_first_pages=41)

# preview first 2 kept pages
pages_and_texts[:2]


Reading pages:   0%|          | 0/1208 [00:00<?, ?it/s]

[{'Page_number': 1,
  'Page_char_count': 745,
  'page_word_count': 117,
  'page_sentence_count_raw': 4,
  'page_token_count': 186,
  'text': 'Special Thanks to Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach College The Children’s Healthy Living Summer Institute – University of Hawai’i at Mānoa Open Educational Resources This text is provided to you as an Open Educational Resource (OER) which you access online. It is designed to give you a comprehensive introduction to human nutrition at no or very nominal cost. It contains both written and graphic text material, intra-text links to other internal material which may aid in understanding topics and concepts, intra-text links to the appendices and glossary for tables and definitions of words, and extra-text links to videos and web material that clarifies and augments topics and concepts. xlii | Acknowledgements'},
 {'Page_number': 2,
  'Page_char_count': 88,
  'page_word_count': 16,
  'page_sentence_count_raw': 3,
  'page

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,Page_number,Page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,745,117,4,186,Special Thanks to Bill Chismar – University of...
1,2,88,16,3,22,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,3,0,0,0,0,
3,4,246,43,2,61,Image by Jim Hollyer / CC BY 4.0 Introduction ...
4,5,1155,176,10,288,Learning Objectives By the end of this chapter...


In [ ]:
df.describe().round(2)

,Page_number,Page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1167.00,1167.00,1167.00,1167.00,1167.00
mean,584.00,1136.94,175.04,14.84,283.86
std,337.03,551.68,86.48,9.51,137.90
min,1.00,0.00,0.00,0.00,0.00
25%,292.50,745.00,114.00,9.00,186.00
50%,584.00,1214.00,189.00,13.00,303.00
75%,875.50,1586.00,241.00,20.00,396.00
max,1167.00,2271.00,393.00,82.00,567.00


In [ ]:
import os
if "COLAB_GPU" in os.environ:
  print("running on Colab GPU")
else:
  print("running on CPU")
  !pip install PyMuPDF #for reading PDFs with python
  !pip install tqdm #for progress bars
  !pip install sentence-transformers #for embedding models
  !pip install accelerate #for quantization model loading
  !pip install bitsandbytes #for quantizing models (less storahge space)
  !pip install flash-attn --no-build-isolation #for faster attention mechanism = faster LLm inference

running on Colab GPU


In [ ]:
from tqdm.auto import tqdm

# ---------- 1️⃣ Split text into chunks ---------- #
def chunk_text(text: str, chunk_size: int = 500) -> list:
    """
    Splits text into chunks of approximately `chunk_size` characters.
    Returns a list of text chunks.
    """
    chunks = []
    current_chunk = ''
    words = text.split()

    for word in words:
        # Check if adding the word exceeds chunk size
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += word + ' '
        else:
            # Store current chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = word + ' '

    # Add the last chunk if not empty
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


# ---------- 2️⃣ Split all PDF pages into chunks ---------- #
def chunk_pdf_pages(pages_and_texts: list, chunk_size: int = 500) -> list[dict]:
    """
    Takes PDF pages (list of dicts) and splits each page text into chunks.
    Returns a list of dicts containing page_number, chunk_index, and chunk_text.
    """
    all_chunks = []
    for page in tqdm(pages_and_texts, desc="Chunking pages"):
        page_number = page["Page_number"]
        page_text = page["text"]

        # Split current page into chunks
        chunks = chunk_text(page_text, chunk_size=chunk_size)

        # Record each chunk with metadata
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i + 1,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) // 4,  # rough token estimate
                "chunk_text": chunk
            })

    return all_chunks


# ---------- 3️⃣ Example usage ---------- #
chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size=500)

print(f"Total chunks: {len(chunked_pages)}")
print(f"First chunk (Page {chunked_pages[0]['page_number']}):\n")
print(chunked_pages[0]['chunk_text'][:400] + "...")


Chunking pages:   0%|          | 0/1167 [00:00<?, ?it/s]

Total chunks: 3253
First chunk (Page 1):

Special Thanks to Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach College The Children’s Healthy Living Summer Institute – University of Hawai’i at Mānoa Open Educational Resources This text is provided to you as an Open Educational Resource (OER) which you access online. It is designed to give you a comprehensive introduction to human nutrition at no or very nominal cost. It conta...


In [ ]:
import pandas as pd
df = pd.DataFrame(chunked_pages)
df.head()

,page_number,chunk_index,chunk_char_count,chunk_word_count,chunk_token_count,chunk_text
0,1,1,497,80,124,Special Thanks to Bill Chismar – University of...
1,1,2,247,37,61,"aid in understanding topics and concepts, intr..."
2,2,1,88,16,22,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
3,4,1,246,43,61,Image by Jim Hollyer / CC BY 4.0 Introduction ...
4,5,1,498,79,124,Learning Objectives By the end of this chapter...


In [ ]:
def _pack_units(units: list[str], max_tokens: int) -> list[list[str]]:
    packs, cur, cur_toks = [], [], 0
    for u in units:
        t = token_len(u)
        if cur and cur_toks + t > max_tokens:
            packs.append(cur)
            cur, cur_toks = [u], t
        else:
            cur.append(u); cur_toks += t
    if cur:
        packs.append(cur)
    return packs

def chunk_text_recursive(text: str, max_tokens: int = 500, overlap_tokens: int = 50) -> list:
    out = []
    paras = split_paragraphs(text)
    for para in paras:
        if token_len(para) <= max_tokens:
            out.append(para)
            continue
        # 1) try sentences
        sents = split_sentences(para)
        if all(token_len(s) <= max_tokens for s in sents):
            for pack in _pack_units(sents, max_tokens):
                out.append(" ".join(pack))
            continue
        # 2) some sentences are huge: fall back to words inside those
        for s in sents:
            if token_len(s) <= max_tokens:
                out.append(s)
            else:
                words = s.split()
                # greedily pack words by character budget approx
                cur, cur_len = [], 0
                char_budget = max_tokens * 4  # inverse of our heuristic
                for w in words:
                    if cur and cur_len + len(w) + 1 > char_budget:
                        out.append(" ".join(cur))
                        cur, cur_len = [w], len(w)
                    else:
                        cur.append(w); cur_len += len(w) + 1
                if cur:
                    out.append(" ".join(cur))
    return with_overlap(out, overlap_tokens=overlap_tokens)


In [ ]:
random.sample(pages_and_texts, k=1)

[{'Page_number': 870,
  'Page_char_count': 977,
  'page_word_count': 147,
  'page_sentence_count_raw': 8,
  'page_token_count': 244,
  'text': 'often mimic their behavior and eating habits. Parents must continue to help their school-aged children and adolescents establish healthy eating habits and attitudes toward food. Their primary role is to bring a wide variety of health-promoting foods into the home, so that their children can make good choices. Learning Activities Technology Note: The second edition of the Human Nutrition Open Educational Resource (OER) textbook features interactive learning activities. These activities are available in the web-based textbook and not available in the downloadable versions (EPUB, Digital PDF, Print_PDF, or Open Document). Learning activities may be used across various mobile devices, however, for the best user experience it is strongly recommended that users complete these activities using a desktop or laptop computer and in Google Chrome. An inte

In [ ]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

# quick test
doc = nlp("This is a sentence. This is another sentence.")
assert len(list(doc.sents)) == 2

from tqdm.auto import tqdm
import pandas as pd

# ---- optional token counter ----
try:
    import tiktoken
    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
    def count_tokens(text):
        return len(enc.encode(text))
except Exception:
    def count_tokens(text):
        # rough fallback: ~1 token ≈ 4 chars
        return len(text) // 4


# ---------- sentence splitting ---------- #
def split_into_sentences(pages, nlp):
    """Return [(page_num, [sent1, sent2,...])]"""
    results = []
    for p in tqdm(pages, desc="Splitting into sentences"):
        doc = nlp(p["text"])
        sents = [s.text.strip() for s in doc.sents if s.text.strip()]
        results.append((p["Page_number"], sents))
    return results


# ---------- sentence-based chunking ---------- #
def chunk_by_sentences(page_sents, chunk_size=10):
    """Group sentences into fixed-size chunks."""
    chunks = []
    for page_num, sents in page_sents:
        for i in range(0, len(sents), chunk_size):
            group = sents[i:i + chunk_size]
            text = " ".join(group)
            chunks.append({
                "page_number": page_num,
                "chunk_index": (i // chunk_size) + 1,
                "chunk_sentence_count": len(group),
                "chunk_word_count": sum(len(s.split()) for s in group),
                "chunk_char_count": len(text),
                "chunk_token_count": count_tokens(text),  # ✅ new field
                "chunk_text": text
            })
    return chunks


# ---------- run it ---------- #
page_sents = split_into_sentences(pages_and_texts, nlp)
chunks_10sent = chunk_by_sentences(page_sents, chunk_size=10)

df_chunks = pd.DataFrame(chunks_10sent)
print(df_chunks.shape)
df_chunks.head()
df.describe().round(2)


Splitting into sentences:   0%|          | 0/1167 [00:00<?, ?it/s]

(1803, 7)


,page_number,chunk_index,chunk_sentence_count,chunk_word_count,chunk_char_count,chunk_token_count
count,1803.00,1803.00,1803.00,1803.00,1803.00,1803.00
mean,597.78,1.41,6.84,113.44,735.67,160.59
std,339.65,0.57,3.28,71.58,450.07,93.06
min,1.00,1.00,1.00,3.00,12.00,3.00
25%,300.50,1.00,4.00,45.00,314.50,80.00
50%,603.00,1.00,8.00,115.00,744.00,161.00
75%,896.00,2.00,10.00,175.00,1125.00,235.50
max,1167.00,3.00,10.00,298.00,1831.00,552.00


In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import numpy as np

# ---------- 1️⃣ Load embedding model ---------- #
# you can choose any; this one is fast and good quality
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# ---------- 2️⃣ Prepare texts to embed ---------- #
texts = [c["chunk_text"] for c in chunks_10sent]  # or df_chunks["chunk_text"].tolist()

# ---------- 3️⃣ Encode with tqdm progress bar ---------- #
embeddings = []
for i in tqdm(range(0, len(texts), 64), desc="Embedding chunks"):
    batch = texts[i:i + 64]
    embs = embedding_model.encode(batch, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=True)
    embeddings.append(embs)

embeddings = np.vstack(embeddings)
print("Embeddings shape:", embeddings.shape)


Embedding chunks:   0%|          | 0/29 [00:00<?, ?it/s]

Embeddings shape: (1803, 384)


In [ ]:
from sentence_transformers import util, SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

# Load the all-mpnet-base-v2 model
embedding_model = SentenceTransformer("all-mpnet-base-v2", device=device)


Running on: cuda


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# --- QUERY → TOP-K RETRIEVAL (dot product / cosine if embeddings were normalized) ---
from sentence_transformers import util
import torch, numpy as np
from time import perf_counter as timer

# Inputs expected:
#   embedding_model : SentenceTransformer (e.g., "all-mpnet-base-v2")
#   embeddings      : np.ndarray of shape (N, D)  (your chunk embeddings)
#   chunks_10sent   : list[dict] with keys: "page_number", "chunk_index", "chunk_text"
#   (or df_chunks DataFrame with same columns)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the SAME embedding model used for the chunks
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)


# 1) Define the query
query = "macronutrients functions"
print(f"Query: {query}")

# 2) Embed the query with the SAME model used for the chunks
query_embedding = embedding_model.encode(
    query,
    convert_to_tensor=True,
    normalize_embeddings=True  # set True if your chunk embeddings were normalized
).to(device)

# Make sure corpus embeddings are a torch tensor on the same device
if isinstance(embeddings, np.ndarray):
    emb_t = torch.from_numpy(embeddings).to(device)
elif isinstance(embeddings, torch.Tensor):
    emb_t = embeddings.to(device)
else:
    raise TypeError("embeddings must be np.ndarray or torch.Tensor")

# 3) Get similarity scores with the dot product (timed)
start_time = timer()
dot_scores = util.dot_score(query_embedding, emb_t)     # shape: [1, N]
end_time = timer()
print(f"Time to get scores on {emb_t.shape[0]} embeddings: {end_time - start_time:.5f} seconds.")

# 4) Get the top-k results
k = 5
top_scores, top_indices = torch.topk(dot_scores.squeeze(0), k=k)
print("Top indices:", top_indices)
print("Top scores:", top_scores)

# 5) (Optional) Pretty print matches with page + snippet
def show_hits(indices, scores, k=5):
    print("\n--- Top results ---")
    for rank, (idx, sc) in enumerate(zip(indices.tolist(), scores.tolist()), start=1):
        if 'df_chunks' in globals():
            row = df_chunks.iloc[idx]
            page_num = int(row["page_number"])
            chunk_idx = int(row["chunk_index"])
            text = row["chunk_text"]
        else:
            row = chunks_10sent[idx]
            page_num = row["page_number"]
            chunk_idx = row["chunk_index"]
            text = row["chunk_text"]
        snippet = (text[:300] + "…") if len(text) > 300 else text
        print(f"{rank}. [page {page_num}, chunk {chunk_idx}]  score={sc:.4f}\n   {snippet}\n")

show_hits(top_indices, top_scores, k=k)

Query: macronutrients functions
Time to get scores on 1803 embeddings: 0.00017 seconds.
Top indices: tensor([   4,    3, 1374,    9,  449], device='cuda:0')
Top scores: tensor([0.6528, 0.6118, 0.5618, 0.5610, 0.5582], device='cuda:0')

--- Top results ---
1. [page 6, chunk 1]  score=0.6528
   Macronutrients Nutrients that are needed in large amounts are called macronutrients. There are three classes of macronutrients: carbohydrates, lipids, and proteins. These can be metabolically processed into cellular energy. The energy from macronutrients comes from their chemical bonds. This chemica…

2. [page 5, chunk 1]  score=0.6118
   Learning Objectives By the end of this chapter, you will be able to: • Describe basic concepts in nutrition • Describe factors that affect your nutritional needs • Describe the importance of research and scientific methods to understanding nutrition What are Nutrients? The foods we eat contain nutri…

3. [page 911, chunk 1]  score=0.5618
   Nutrient Adult Males A

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# same model used for both documents and queries
# Use the same model that generated the 'embeddings' variable
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# encode your chunks once and store them
# embeddings = embedding_model.encode(chunk_texts, normalize_embeddings=True)

# later, when a user asks a query:
query = "macronutrient functions"
query_embedding = embedding_model.encode(
    query,
    convert_to_tensor=True,
    normalize_embeddings=True
)

# compute similarity (cosine == dot product since normalized)
dot_scores = util.dot_score(query_embedding, torch.from_numpy(embeddings).to("cuda"))
top_k = torch.topk(dot_scores.squeeze(0), k=5)

In [57]:
from sentence_transformers import util, SentenceTransformer
import torch
from time import perf_counter as timer
from typing import List, Dict

def retrieve_relevant_resources(
    query: str,
    embeddings,
    model: SentenceTransformer,
    n_resources_to_return: int = 5,
    print_time: bool = True
):
    """
    Embeds a query using the same model as your document embeddings and returns
    the top-k similarity scores and indices.

    Args:
        query (str): The user query or search text.
        embeddings (torch.Tensor or np.ndarray): Precomputed embeddings of your chunks.
        model (SentenceTransformer): Embedding model (same used for chunks).
        n_resources_to_return (int): Number of top results to return.
        print_time (bool): Whether to print how long retrieval took.

    Returns:
        scores (torch.Tensor): Similarity scores (cosine if normalized)
        indices (torch.Tensor): Indices of top results
    """

    # ---- 1. Embed the query ---- #
    query_embedding = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

    # Ensure chunk embeddings are on the same device as the query embedding
    if isinstance(embeddings, np.ndarray):
        embeddings = torch.from_numpy(embeddings).to(query_embedding.device)
    elif isinstance(embeddings, torch.Tensor):
        embeddings = embeddings.to(query_embedding.device)
    else:
        raise TypeError("embeddings must be np.ndarray or torch.Tensor")

    # ---- 2. Compute similarity (dot product == cosine if normalized) ---- #
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to compute scores on {len(embeddings)} embeddings: "
              f"{end_time - start_time:.5f} seconds.")

    # ---- 3. Get top-k matches ---- #
    scores, indices = torch.topk(input=dot_scores, k=n_resources_to_return)
    return scores, indices


def print_top_results_and_scores(
    query: str,
    embeddings: torch.Tensor,
    pages_and_chunks: List[Dict],
    model: SentenceTransformer,
    top_k: int = 5
):
    """
    Uses `retrieve_relevant_resources()` to get top results
    and prints them with page numbers, chunk indices, and snippet previews.
    """
    scores, indices = retrieve_relevant_resources(
        query=query,
        embeddings=embeddings,
        model=model,
        n_resources_to_return=top_k
    )

    print(f"\nQuery: {query}")
    print("-" * 70)
    for rank, (idx, score) in enumerate(zip(indices.tolist(), scores.tolist()), start=1):
        chunk = pages_and_chunks[idx]
        snippet = chunk["chunk_text"][:300].replace("\n", " ")
        print(f"{rank}. [Page {chunk['page_number']}, Chunk {chunk['chunk_index']}] "
              f"Score: {score:.4f}\n   {snippet}...\n")

In [ ]:
import pandas as pd

# make a shallow copy to avoid modifying your main df
df_export = df_chunks.copy()
df_export["embedding"] = [",".join(map(str, emb)) for emb in embeddings]

csv_path = "/content/chunks_with_embeddings.csv"
df_export.to_csv(csv_path, index=False)
print(f"✅ Embeddings exported to {csv_path}")


✅ Embeddings exported to /content/chunks_with_embeddings.csv


In [ ]:
!pip install spacy


In [ ]:
df = pd.DataFrame(chunks_10sent)
df.describe().round(2)

,page_number,chunk_index,chunk_sentence_count,chunk_word_count,chunk_char_count,chunk_token_count
count,1803.00,1803.00,1803.00,1803.00,1803.00,1803.00
mean,597.78,1.41,6.84,113.44,735.67,160.59
std,339.65,0.57,3.28,71.58,450.07,93.06
min,1.00,1.00,1.00,3.00,12.00,3.00
25%,300.50,1.00,4.00,45.00,314.50,80.00
50%,603.00,1.00,8.00,115.00,744.00,161.00
75%,896.00,2.00,10.00,175.00,1125.00,235.50
max,1167.00,3.00,10.00,298.00,1831.00,552.00


In [59]:
print_top_results_and_scores(
    query=query,
    embeddings=embeddings,
    pages_and_chunks=chunks_10sent,
    model=embedding_model
)

[INFO] Time taken to compute scores on 1803 embeddings: 0.00011 seconds.

Query: symptoms of pellegra
----------------------------------------------------------------------
1. [Page 566, Chunk 2] Score: 0.5319
   Niacin deficiency is commonly known as pellagra and the symptoms include fatigue, decreased appetite, and indigestion. These symptoms are then commonly followed by the four D’s: diarrhea, dermatitis, dementia, and sometimes death. Figure 9.12 Conversion of Tryptophan to Niacin Water-Soluble Vitamins...

2. [Page 12, Chunk 1] Score: 0.3801
   Vitamins Major Functions Water-soluble Thiamin (B1) Coenzyme, energy metabolism assistance Riboflavin (B2 ) Coenzyme, energy metabolism assistance Niacin (B3) Coenzyme, energy metabolism assistance Pantothenic acid (B5) Coenzyme, energy metabolism assistance Pyridoxine (B6) Coenzyme, amino acid synt...

3. [Page 178, Chunk 3] Score: 0.3385
   Symptoms, including nausea, muscle cramps, confusion, dizziness, and in severe cases, coma and dea

In [ ]:
import random
df_chunks = pd.DataFrame(chunks_10sent)
print(df_chunks.shape)
df_chunks.head()

(1803, 6)


,page_number,chunk_index,chunk_sentence_count,chunk_word_count,chunk_char_count,chunk_text
0,1,1,4,117,745,Special Thanks to Bill Chismar – University of...
1,2,1,3,16,88,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,4,1,1,43,246,Image by Jim Hollyer / CC BY 4.0 Introduction ...
3,5,1,10,176,1155,Learning Objectives By the end of this chapter...
4,6,1,10,149,987,Macronutrients Nutrients that are needed in la...


In [58]:
query = "symptoms of pellegra"
scores, indices = retrieve_relevant_resources(query=query, embeddings=embeddings, model=embedding_model)
scores, indices

[INFO] Time taken to compute scores on 1803 embeddings: 0.00014 seconds.


(tensor([0.5319, 0.3801, 0.3385, 0.3181, 0.3125], device='cuda:0'),
 tensor([ 854,   13,  256, 1742, 1735], device='cuda:0'))

In [65]:
!pip install nbformat

import nbformat

path = '/content/RAG_Pipeline.ipynb'  # change to your notebook path
nb = nbformat.read(path, as_version=4)

# Remove problematic widget metadata
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Save cleaned notebook
nbformat.write(nb, path)
print('✅ Cleaned notebook metadata. You can now push to GitHub.')

✅ Cleaned notebook metadata. You can now push to GitHub.
